# XRD angles
conventions are from Busing Levy Acta Cryst. (1967) 22, 457

In [88]:
# Sr3Co2Fe24O41
import numpy as np
from scipy.spatial.transform import Rotation as R



np.set_printoptions(suppress=True)
np.set_printoptions(precision=4)

# Hexaferrite-multiferroic Sr3Co2Fe24O41
# Sr3Co2Fe24O41  see for instance Phys. Rev. Lett. 100, 094444 (2019)
#  P63/mmc
# this corresponds to lattice parameters a = b = 5.87 Å and c = 52.07 Å
# 
# 

#lattice parameters
a =  5.87 # in A
b =  5.87
c =  52.07
alpha = 90 # in deg
beta  = 90
gamma = 120



# U: Orientation matrix. Describes how the sample is glued on the diffr. 
# With all diffractometer angles =  0
# y axes of the diffractometer is pointig along the beam and z is 
#out of the scattering plane (see fig 1 Acta Cryst.(1967).22, 457)

#U = np.identity(3) # Sample glued with c axis along z - diff axis for all angles = 0

# usually crystal is cut so a-axis is out-of-plane, and c-axis in plane
#U = R.from_rotvec(-np.pi/2 * np.array([0, 1, 0])).as_matrix() #Sample glued with c axis along x - diff axis for all angles = 0


#Reciprocal lattice parameters
# the 2pi factor is not here so |q| is 1/d , d miller plane distance. If you put here hte 2pi the |q| is 2pi/d and formula for 
# bragg condition needs to be modified below


ka = 2/(np.sqrt(3)*a)
kb = 2/(np.sqrt(3)*a)
kc = 1/c
kalpha = 90 # deg
kbeta  = 90
kgamma = 30







#
h = 0
k = 0
l = 4





#hv

hv = 0.650 # in keV


In [89]:
#Miller vector
Mv=np.array([h, k, l])


# B: Conversion matrix from reciprocal vectors to orth. cartesian coordinates defined as
# x parallel to ka
# y in the plane of ka, kb
# z perp to the plane of ka, kb
B = np.array([[ka, kb*np.cos(kgamma*np.pi/180), kc*np.cos(kbeta*np.pi/180)                        ],
              [ 0, kb*np.sin(kgamma*np.pi/180),-kc*np.sin(kbeta*np.pi/180)*np.cos(alpha*np.pi/180)],      #not that clear why there is alpha and not kalpha
              [ 0,             0,                            1/c                                  ] ])    #not that clear why there is 1/c and not -kc*np.sin(kbeta*np.pi/180)*np.SIN(alpha*np.pi/180)]

print(B)


hc   = B.dot(Mv)
hphi = U.dot(hc)

print(hphi)

q = np.linalg.norm(hc)
qph = 0.508 * hv #in A-1
lambd = 2*np.pi/qph #in A 
delta=0.0001

print("q norm (1 / Miller Planes distance): %8.3f" % q)
print("q*lambd/2 : %8.3f" % ( q*lambd/2 ) )


[[ 0.1967  0.1704  0.    ]
 [ 0.      0.0984 -0.    ]
 [ 0.      0.      0.0192]]
[ 0.     -0.      0.0768]
q norm (1 / Miller Planes distance):    0.077
q*lambd/2 :    0.731


In [90]:
#Angles in bisecting geometry omega=0
Phi   = np.arctan(hphi[1]/(hphi[0]+delta)) *180/np.pi
Chi   = np.arctan(hphi[2]/ np.sqrt(hphi[0]**2 + hphi[1]**2))*180/np.pi
Omega = 0
Theta = np.arcsin(lambd*q / 2)*180/np.pi


print("Phi : %8.3f" % Phi)
print("Chi : %8.3f" % Chi)
print("Omega : %8.3f" % Omega) 
print("Omega+theta : %8.3f" % (Omega+Theta)) 
print("2Theta : %8.3f" % (2*Theta))

Phi :   -0.000
Chi :   90.000
Omega :    0.000
Omega+theta :   46.960
2Theta :   93.920


In [77]:
# Angles in Chi =   90 geometry are:
Phi = np.arctan(-hphi[0]/(hphi[1]+delta))*180/np.pi
Chi = 90
Omega = np.arctan( np.sqrt(hphi[0]**2 + hphi[1]**2)/ (hphi[2]+delta))*180/np.pi
Theta = np.arcsin(lambd*q / 2)*180/np.pi


print("Phi : %8.3f" % Phi)
print("Chi : %8.3f" % Chi)
print("Omega : %8.3f" % Omega) 
print("Omega+theta : %8.3f" % (Omega+Theta)) 
print("2Theta : %8.3f" % (2*Theta))

Phi :   -0.000
Chi :   90.000
Omega :    0.000
Omega+theta :   42.057
2Theta :   84.114


In [63]:


##option 1. Build R0 from bisecting or Chi=90 geometry depending on the last you run
PHI0   = R.from_rotvec(-np.pi/180* Phi   * np.array([0, 0, 1])).as_matrix()  
X0     = R.from_rotvec(-np.pi/180* Chi   * np.array([0, 1, 0])).as_matrix()
OMEGA0 = R.from_rotvec(-np.pi/180* Omega * np.array([0, 0, 1])).as_matrix()




# Option 2. Build R0 from hphi and h0phi(!=hphi). 
# h0phi is a user specified lattice direction that at Psi=0 is placed in the scattering plane

h0=np.array([0,0,1])
h0c=B.dot(h0)
h0phi = U.dot(h0c)

t1phi =hphi/ np.linalg.norm(hphi)             
t2phi = h0phi - h0phi.dot(t1phi)*t1phi         
t2phi = t2phi/np.linalg.norm(t2phi)  
t3phi = np.cross(t1phi,t2phi)
#t3phi = t3phi/np.linalg.norm(t3phi)  

#For option 1 run
#R0 = OMEGA0.dot(X0.dot(PHI0))

#For option 2 run
R0 = np.array([t1phi, t2phi, t3phi])

#
# Now rotate around the scatering vector of psi to get the (new for op. 1) angles
Psi= .1
PSI0 = R.from_rotvec(-np.pi/180* Psi * np.array([1, 0, 0])).as_matrix()

Rp= PSI0.dot(R0)

Chi_psi   = np.arctan( np.sqrt(Rp[2,0]**2 + Rp[2,1]**2)/ Rp[2,2] )*180/np.pi
Phi_psi   = np.arctan( -Rp[2,1]/ -Rp[2,0])*180/np.pi
Omega_psi = np.arctan( -Rp[1,2]/(Rp[0,2]+delta))*180/np.pi

print("Phi_psi : %8.3f" % Phi_psi)
print("TLT_psi : %8.3f" % Chi_psi)
print("Omega_psi : %8.3f" % Omega_psi) 
print("Omega_psi+theta : %8.3f" % (Omega_psi+Theta) )
print("2Theta : %8.3f" % (2*Theta))


Phi_psi :      nan
TLT_psi :      nan
Omega_psi :      nan
Omega_psi+theta :      nan
2Theta :   84.114


<ipython-input-63-370bbd4c3009>:18: RuntimeWarning: invalid value encountered in true_divide
  t2phi = t2phi/np.linalg.norm(t2phi)


In [205]:
t1phi

array([0.    , 0.7071, 0.7071])

In [206]:
t2phi

array([ 0.    , -0.7071,  0.7071])

In [203]:
t3phi

array([-1.,  0.,  0.])

In [207]:
R0

array([[ 0.    ,  0.7071,  0.7071],
       [ 0.    , -0.7071,  0.7071],
       [ 1.    ,  0.    , -0.    ]])

In [209]:
np.linalg.inv(R0)

array([[ 0.    ,  0.    ,  1.    ],
       [ 0.7071, -0.7071, -0.    ],
       [ 0.7071,  0.7071, -0.    ]])

In [246]:
hphi

array([ 0.    , -0.    , -0.0916])